In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target

Cloning into 'Recommender-Systems-Challenge-2022'...
remote: Enumerating objects: 619, done.
remote: Counting objects: 100% (215/215), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 619 (delta 107), reused 139 (delta 52), pack-reused 404
Receiving objects: 100% (619/619), 106.18 MiB | 21.12 MiB/s, done.
Resolving deltas: 100% (279/279), done.
Updating files: 100% (254/254), done.


In [2]:
URM_all = load_urm()

#URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
#URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.85)
URM_train = sps.load_npz('Dataset/Split/URM_train.npz')
URM_test =  sps.load_npz('Dataset/Split/URM_test.npz')
URM_validation = sps.load_npz('Dataset/Split/URM_validation.npz')

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1541 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 958 ( 2.3%) Users that have less than 1 test interactions


In [3]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_all)
recommender_SLIMElasticNet.fit(topK=585, l1_ratio=0.009929794168572358, alpha=0.0018687691383859428)

SLIMElasticNetRecommender: Processed 2608 (10.6%) in 5.00 min. Items per second: 8.69
SLIMElasticNetRecommender: Processed 5542 (22.6%) in 10.00 min. Items per second: 9.23
SLIMElasticNetRecommender: Processed 8429 (34.4%) in 15.00 min. Items per second: 9.36
SLIMElasticNetRecommender: Processed 11261 (46.0%) in 20.00 min. Items per second: 9.38
SLIMElasticNetRecommender: Processed 14095 (57.5%) in 25.00 min. Items per second: 9.39
SLIMElasticNetRecommender: Processed 17023 (69.5%) in 30.00 min. Items per second: 9.46
SLIMElasticNetRecommender: Processed 19828 (80.9%) in 35.01 min. Items per second: 9.44
SLIMElasticNetRecommender: Processed 22736 (92.8%) in 40.01 min. Items per second: 9.47
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 43.10 min. Items per second: 9.48


In [4]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

recommender_RP3beta = RP3betaRecommender(URM_all)
recommender_RP3beta.fit(alpha=0.6949339074999242, beta=0.2853519331891143, topK=59, implicit=True)
#recommender_RP3beta.fit(alpha=0.9440960931970539, beta=0.23362882450834893, topK=54, implicit=True)

RP3betaRecommender: Similarity column 24507 (100.0%), 1003.61 column/sec. Elapsed time 24.42 sec


In [5]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

alpha = 0.5
new_similarity = (1 - alpha) * recommender_SLIMElasticNet.W_sparse + alpha * recommender_RP3beta.W_sparse

recommender = ItemKNNCustomSimilarityRecommender(URM_all)
recommender.fit(new_similarity)

In [6]:
#from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

#recommender_EaseR = EASE_R_Recommender(URM_all)
#recommender_EaseR.fit(topK=None, normalize_matrix=False, l2_norm=157.86876317814773)

In [7]:
from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
        
    def fit(self, norm, alpha = 0.5):

        self.alpha = alpha
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * (1-self.alpha)

        return item_weights

In [8]:
#recommender = DifferentLossScoresHybridRecommender(URM_all, recommender_SLIMElasticNet_RP3Beta, recommender_EaseR)
#recommender.fit(norm=1, alpha = 1.0)

## Submissions


In [9]:
test_users = pd.read_csv('Dataset/data_target_users_test.csv')

In [10]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user, cutoff=10))

In [11]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('Submissions/Submission_SLIM_EN_rp3Beta_Similarity.csv', index=False)